In [1]:
import argparse
from datetime import datetime
import json
import logging
import os
import math
import sys
import time
import topologyqueries,linkedlist_old,update_topology_old,spanning_tree_old

from gridappsd import GridAPPSD, DifferenceBuilder, utils, topics as t
from gridappsd.topics import simulation_input_topic, simulation_log_topic, simulation_output_topic

In [2]:

#model_mrid = sim_request["power_system_config"]["Line_name"]
#model_mrid = "_C125761E-9C21-4CA9-9271-B168150DE276" #ieee13training
model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node
#model_mrid = "_AAE94E4A-2465-6F5E-37B1-3E72183A4E44" #test9500
#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" #ieee13assets
#model_mrid="_C1C3E687-6FFD-C753-582B-632A27E28507" # IEEE 123

requestedTime=1570041125



In [3]:
global BaseConnDict,BaseTermDict,TermList, NodeList
global XfmrKeys,XfmrDict,SwitchKeys,SwitchDict,DG_query

os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'

gapps = GridAPPSD()
assert gapps.connected

In [4]:
# RUN DEMO 9500 NODE SIMULATION
import Run9500NodeDemo as r
simulation_id = r.simulation.simulation_id
time.sleep(15)
print(simulation_id)
# Note: Restart and clear notebook kernel to re-run simulation

244248565


In [5]:
# Query for ACLineSegment, Transformerend, Switches, Breakers, Reclosers, Fuses, Sectionalisers,SynchronousMachine,TopologicalNode
topologyqueries.getallqueries(gapps,model_mrid)
Line_query=topologyqueries.Line_query
XfmrDict=topologyqueries.XfmrDict
XfmrKeys=topologyqueries.XfmrKeys
SwitchDict=topologyqueries.SwitchDict
SwitchKeys=topologyqueries.SwitchKeys
DG_query=topologyqueries.DG_query
Node_query=topologyqueries.Node_query

In [6]:
# Build Linknet Lists from ACLineSegment,Transformers,DGs and Nodes
[ConnNodeDict,TerminalsDict,TermList,NodeList]=linkedlist_old.build_linked_list(Line_query,XfmrDict,XfmrKeys,DG_query,Node_query)

# Stash a copy of base dictionary
BaseConnDict = json.dumps(ConnNodeDict)
BaseTermDict = json.dumps(TerminalsDict)

Processed  3909 line objects in  0.025604422999999987 seconds
Processed  1304 transformer objects in  0.00844353199999992 seconds
Processed  9 generator objects in  0.0001117350000000572 seconds
Processed  4 missing nodes in  0.005028448000000019 seconds


In [7]:
#IF SIMULATION ID IS SPECIFIED, GET TOPOLOGY FOR REQUESTED TIME FOR SPECIFIED TIMESTAMP

equipment_dict = {}
measurement_dict = {}
meas_map={}

#Query for switch position measurement mRIDs
request = {"modelId": model_mrid,
           "requestType": "QUERY_OBJECT_MEASUREMENTS",
           "resultFormat": "JSON",
           "objectType": "LoadBreakSwitch"
           }

response = gapps.get_response(t.REQUEST_POWERGRID_DATA,request,timeout=15)

for measurement in response["data"]:
    if measurement["type"] == "Pos":
        measid = measurement["measid"]
        measurement_dict[measid] = measurement
        meas_map[measid] = measurement["eqid"]
        SwitchDict[measurement["eqid"]]["measid"] = measid

In [8]:
topic = "goss.gridappsd.process.request.data.timeseries" # Specify Timeseries API GridAPPS-D topic

# Query for a particular set of measurments
message = {
    "queryMeasurement":"simulation",
    "queryFilter":{"simulation_id": str(simulation_id),
        "startTime": str(round(requestedTime-1)),
        "endTime": str(round(requestedTime+3)),
        "measurement_mrid": list(measurement_dict.keys())
                  },
        "responseFormat":"JSON" }


influx_response = gapps.get_response(t.TIMESERIES, message) # Pass API call

counter=0
while not influx_response['data'] and counter <5:
    influx_response = gapps.get_response(topic, message) # Pass API call
    print('Waiting 10 sec for data to be written to Timeseries Database')
    time.sleep(10)
    counter=counter+1
    if counter==5: print("No Timeseries data found. Returning default topology")

Waiting 10 sec for data to be written to Timeseries Database
Waiting 10 sec for data to be written to Timeseries Database


KeyboardInterrupt: 

In [ ]:
for measurement in influx_response["data"]:
    measid = measurement["measurement_mrid"]
    time = measurement["time"]
    eqid = meas_map[measid]
    SwitchDict[eqid]["open"] = measurement['value']

In [10]:
# Process Switch Topology - run at each switch change - merges topology nodes across closed switches

[TerminalsDict,ConnNodeDict]=update_topology_old.topology_update(BaseConnDict,BaseTermDict,SwitchDict,SwitchKeys,TermList)

# Build Spanning Tree from Xfmrs & DGs


Processed  107 switch objects in  0.0003514700000000204 seconds


In [11]:
ConnNodeDict['_EEB13278-5AE5-4401-BBF6-5F3A1CC48080']

{'name': 'm2001-mt2',
 'node': 5288,
 'list': 10432,
 'tpid': '_B37DB28C-93CC-403C-9EEB-7231CB4B2F39',
 'der': ['microturb-2'],
 'nomv': 0}

In [17]:

[Tree,TotalNodes]=spanning_tree_old.generate_spanning_tree(XfmrKeys,XfmrDict,ConnNodeDict,TerminalsDict,TermList,NodeList,DG_query)

Processed topology from substation transformer  hvmv69_11sub3_T1  with  2264  buses
Processed topology from substation transformer  hvmv69_11sub1_T1  with  1151  buses
Processed topology from substation transformer  hvmv69_11sub2_T1  with  1859  buses
Processed  3 topology trees containing  5274  buses in  0.17925825400000006 seconds
Processed  0  more islands containing  0  buses in  0.0011866249999998857 seconds
 
 


In [12]:
SwitchDict

{'_2988C4A4-4F1A-4E2F-8947-E79A582DE021': {'name': '2002200004641085_sw',
  'bus1': 'd5587291-3_int',
  'bus2': 'q14734',
  'term1': '_78F020AA-60BD-4747-9319-E9011297ABD4',
  'term2': '_E8CE7965-14DA-4050-9726-216659627024',
  'node1': '_05379F44-A2FE-4E0E-B1A4-7B54E14225C0',
  'node2': '_77816FBC-36C7-4D4D-82C4-8FBA714BDFFF',
  'tpnode1': '_F96AB09A-B23D-4178-9E2A-833872F0F1E2',
  'tpnode2': '_FE66D827-776C-4C6A-83C6-22C29AD337BD',
  'open': 1,
  'measid': '_819c1eca-6480-4844-a0c4-7c709c23cbc6'},
 '_1CB6F0F5-08B3-456C-9D7D-9BCC866A2FB4': {'name': '2002200004868472_sw',
  'bus1': 'd5563942-4_int',
  'bus2': 'q16483',
  'term1': '_B3D237FF-2CFB-4A83-929B-896071F0781A',
  'term2': '_4F32AD5B-92A9-49C7-8CCA-7864A19D1307',
  'node1': '_74CC8623-F187-44DD-A335-E8E0318DCDBF',
  'node2': '_9A3487F5-DB44-4F62-BE3A-D3CCEA35E9E5',
  'tpnode1': '_1C5C953D-1183-49CE-9032-09728677BADC',
  'tpnode2': '_1CFC5FA1-8079-4B7E-8A97-6D3DEDDF50BF',
  'open': 1,
  'measid': '_f58e97b6-ccd5-428f-936b-8ca21b

In [13]:
ConnNodeDict['_03BBA6C7-58C4-4021-8CA4-9BED6D3825C9']

{'lines': ['ln5806917-1'],
 'line_ids': ['_F14487F1-601C-418B-B038-8D5BABC42020'],
 'name': 'e182723',
 'node': 1448,
 'list': 2009,
 'tpid': '_53B7FFEC-1AC9-4CB3-A602-6CCBC0A099F5',
 'nomv': 0}

In [14]:
ConnNodeDict['_9C3C908A-9FD7-4E58-930F-73273D42F3E9']

{'lines': ['ln5746546-1'],
 'line_ids': ['_25C1A76E-C485-4517-ABDC-AE97D30B1ED0'],
 'name': 'd5746546-1_int',
 'node': 1261,
 'list': 1710,
 'tpid': '_F2B98C16-D1DB-4A72-B6BD-5BCAE017ACF8',
 'nomv': 0}